In [1]:
using CSV
using DataFrames
using Random
using Base.Iterators: repeated, partition
import StatsBase.sample, StatsBase.Weights
using Flux
using Flux: onehot, onecold, onehotbatch, crossentropy, throttle, params
using Zygote
using BSON, JLD2, Statistics
using CUDA
using XGBoost


In [2]:


# Load data
data = CSV.read("G:/LLM/own60k.csv", DataFrame)

# Set the random seed for reproducibility
Random.seed!(123)

# Determine the split ratios
train_ratio = 0.9
validation_ratio = 0.04
test_ratio = 0.06

# Calculate the split indices
n_samples = nrow(data)
train_index = Int(round(train_ratio * n_samples))
validation_index = Int(round((train_ratio + validation_ratio) * n_samples))

# Shuffle the data
shuffle!(data)

# Split the data
train_data = data[1:train_index, :]
validation_data = data[train_index+1:validation_index, :]
test_data = data[validation_index+1:end, :]

# Function to parse Float32 arrays from strings
function parse_float32_array(str)
    # Remove "Float32[" prefix and "]" suffix
    clean_str = strip(str, ['F', 'l', 'o', 'a', 't', '3', '2', '[', ']'])
    # Split by commas and parse each value as Float32
    return Float32[parse(Float32, replace(x, "f" => "e")) for x in split(clean_str, ", ")]
end

# Prepare the matrices for the models
X_train = [parse_float32_array(row[end]) for row in eachrow(train_data)]
y_train = train_data.stars
X_valid = [parse_float32_array(row[end]) for row in eachrow(validation_data)]
y_valid = validation_data.stars
X_test = [parse_float32_array(row[end]) for row in eachrow(test_data)]
y_test = test_data.stars

# Convert to matrices
X_train = hcat(X_train...)'
X_valid = hcat(X_valid...)'
X_test = hcat(X_test...)'


3600×300 adjoint(::Matrix{Float32}) with eltype Float32:
 0.0781837  0.0406476  -0.0353282  …  -0.00141612   0.025835   -0.0234873
 0.0804505  0.0424143  -0.033293      -0.00340486   0.0196931  -0.0229729
 0.0818618  0.0517287  -0.0289263      0.00323588   0.0246126  -0.0322359
 0.0785283  0.0411878  -0.0356211     -0.000431011  0.0278983  -0.0240961
 0.0771156  0.0430582  -0.0336129     -0.00147919   0.0268414  -0.0281207
 0.0786086  0.0455124  -0.0313527  …  -0.00831156   0.023752   -0.02752
 0.0821883  0.044521   -0.0166063      0.00134261   0.0293592  -0.0250655
 0.0812202  0.0448783  -0.0306634      0.000305143  0.0299941  -0.0253616
 0.0763234  0.0386637  -0.0339257     -0.00348777   0.0236798  -0.0229157
 0.078516   0.0420292  -0.0324027     -0.00252204   0.0232836  -0.0268073
 0.0824733  0.0410133  -0.0263026  …  -0.00121429   0.0284368  -0.0243518
 0.0789983  0.0439928  -0.031346       0.00044325   0.0286855  -0.0257789
 0.0766955  0.0389441  -0.0360676     -0.00137574   0.022

In [3]:

# Check if GPU is available and set the device accordingly
use_cuda = true
if use_cuda && CUDA.functional()
    device = gpu
    @info "Training on GPU"
else
    device = cpu
    @info "Training on CPU"
end

# Transpose and move X_train to the correct device
X_train1 = transpose(X_train) |> device
batch_size = 32
X_valid1 = transpose(X_valid) |> device
X_test1 = transpose(X_test) |> device

# Convert y_train to onehot encoding and move to the correct device
y_train1 = Flux.onehotbatch(y_train, 1:5) |> device
y_valid1 = Flux.onehotbatch(y_valid, 1:5) |> device
y_test1 = Flux.onehotbatch(y_test, 1:5) |> device

# Create DataLoader and move to the correct device
loader = Flux.DataLoader((X_train1, y_train1), batchsize=batch_size, shuffle=true)

# Define the MLP model and move to the correct device
model = Chain(
    Dense(300, 128, relu),
    Dense(128, 64, relu),
    Dense(64, 5),
    softmax
) |> device

# Define the loss function
loss(model, X, y) = Flux.crossentropy(model(X), y)

# Setup the optimizer
opt = Flux.setup(Adam(0.0004), model)

# Function to calculate accuracy
accuracy(X, y) = mean(onecold(model(X)) .== onecold(y))

# Training with validation
best_val_accuracy = 0.0
best_model_params = nothing
last_imp=0
for epoch in 1:491
    Flux.train!(loss, model, loader, opt)
    val_accuracy = accuracy(X_valid1, y_valid1)
    println("Epoch $epoch: Validation Accuracy = $val_accuracy")
    last_imp+=1
    if val_accuracy > best_val_accuracy
        best_val_accuracy = val_accuracy
        model_params = cpu(Flux.state(model))
        BSON.@save "bestmlp3layersown.bson" model_params
        last_imp=0
    end
    if last_imp>10
        break
    end
end



# Evaluate on the test set
test_accuracy = accuracy(X_test1, y_test1)
println("MLP Accuracy on Test Set: ", test_accuracy)


[ Info: Training on GPU


Epoch 1: Validation Accuracy = 0.47375
Epoch 2: Validation Accuracy = 0.47375
Epoch 3: Validation Accuracy = 0.4741666666666667
Epoch 4: Validation Accuracy = 0.49625
Epoch 5: Validation Accuracy = 0.49916666666666665
Epoch 6: Validation Accuracy = 0.505
Epoch 7: Validation Accuracy = 0.5079166666666667
Epoch 8: Validation Accuracy = 0.49916666666666665
Epoch 9: Validation Accuracy = 0.5258333333333334
Epoch 10: Validation Accuracy = 0.5279166666666667
Epoch 11: Validation Accuracy = 0.47541666666666665
Epoch 12: Validation Accuracy = 0.5258333333333334
Epoch 13: Validation Accuracy = 0.5470833333333334
Epoch 14: Validation Accuracy = 0.53625
Epoch 15: Validation Accuracy = 0.5483333333333333
Epoch 16: Validation Accuracy = 0.5566666666666666
Epoch 17: Validation Accuracy = 0.5533333333333333
Epoch 18: Validation Accuracy = 0.55875
Epoch 19: Validation Accuracy = 0.55375
Epoch 20: Validation Accuracy = 0.54125
Epoch 21: Validation Accuracy = 0.5316666666666666
Epoch 22: Validation Accu

In [4]:


# Check if GPU is available and set the device accordingly
use_cuda = true
if use_cuda && CUDA.functional()
    device = gpu
    @info "Training on GPU"
else
    device = cpu
    @info "Training on CPU"
end

# Transpose and move X_train to the correct device
X_train1 = transpose(X_train) |> device
batch_size = 32
X_valid1 = transpose(X_valid) |> device
X_test1 = transpose(X_test) |> device

# Convert y_train to onehot encoding and move to the correct device
y_train1 = Flux.onehotbatch(y_train, 1:5) |> device
y_valid1 = Flux.onehotbatch(y_valid, 1:5) |> device
y_test1 = Flux.onehotbatch(y_test, 1:5) |> device

# Create DataLoader and move to the correct device
loader = Flux.DataLoader((X_train1, y_train1), batchsize=batch_size, shuffle=true)

# Define the MLP model and move to the correct device
model = Chain(
    Dense(300, 128, relu),
    Dense(128, 64, relu),
    Dense(64, 5),
    softmax
) |> device

# Define the loss function
loss(model, X, y) = Flux.crossentropy(model(X), y)

# Setup the optimizer
opt = Flux.setup(AdaGrad(0.004), model)

# Function to calculate accuracy
accuracy(X, y) = mean(onecold(model(X)) .== onecold(y))

# Training with validation
best_val_accuracy = 0.0
best_model_params = nothing
last_imp=0
for epoch in 1:491
    Flux.train!(loss, model, loader, opt)
    val_accuracy = accuracy(X_valid1, y_valid1)
    println("Epoch $epoch: Validation Accuracy = $val_accuracy")

    if val_accuracy > best_val_accuracy
        best_val_accuracy = val_accuracy
        model_params = cpu(Flux.state(model))
        BSON.@save "bestmlp3layersownada.bson" model_params   
        last_imp=0
    end
    if last_imp>10
        break
    end
end



# Evaluate on the test set
test_accuracy = accuracy(X_test1, y_test1)
println("MLP Accuracy on Test Set: ", test_accuracy)


[ Info: Training on GPU


Epoch 1: Validation Accuracy = 0.47375
Epoch 2: Validation Accuracy = 0.47375
Epoch 3: Validation Accuracy = 0.47375
Epoch 4: Validation Accuracy = 0.47375
Epoch 5: Validation Accuracy = 0.47375
Epoch 6: Validation Accuracy = 0.47375
Epoch 7: Validation Accuracy = 0.47375
Epoch 8: Validation Accuracy = 0.47375
Epoch 9: Validation Accuracy = 0.47375
Epoch 10: Validation Accuracy = 0.47375
Epoch 11: Validation Accuracy = 0.47375
Epoch 12: Validation Accuracy = 0.47375
Epoch 13: Validation Accuracy = 0.47375
Epoch 14: Validation Accuracy = 0.47375
Epoch 15: Validation Accuracy = 0.47375
Epoch 16: Validation Accuracy = 0.47375
Epoch 17: Validation Accuracy = 0.47375
Epoch 18: Validation Accuracy = 0.47375
Epoch 19: Validation Accuracy = 0.47375
Epoch 20: Validation Accuracy = 0.47375
Epoch 21: Validation Accuracy = 0.47375
Epoch 22: Validation Accuracy = 0.47375
Epoch 23: Validation Accuracy = 0.47375
Epoch 24: Validation Accuracy = 0.4741666666666667
Epoch 25: Validation Accuracy = 0.4741

In [5]:
using Flux
using Flux: onehot, onecold, onehotbatch, crossentropy, throttle, params
using Zygote
using BSON, JLD2, Statistics
using CUDA
using Random
using Base.Iterators: repeated, partition
import StatsBase.sample, StatsBase.Weights

# Check if GPU is available and set the device accordingly
use_cuda = true
if use_cuda && CUDA.functional()
    device = gpu
    @info "Training on GPU"
else
    device = cpu
    @info "Training on CPU"
end

# Transpose and move X_train to the correct device
X_train1 = transpose(X_train) |> device
batch_size = 32
X_valid1 = transpose(X_valid) |> device
X_test1 = transpose(X_test) |> device

# Convert y_train to onehot encoding and move to the correct device
y_train1 = Flux.onehotbatch(y_train, 1:5) |> device
y_valid1 = Flux.onehotbatch(y_valid, 1:5) |> device
y_test1 = Flux.onehotbatch(y_test, 1:5) |> device

# Create DataLoader and move to the correct device
loader = Flux.DataLoader((X_train1, y_train1), batchsize=batch_size, shuffle=true)

# Define the MLP model and move to the correct device
model = Chain(
    Dense(300, 128, relu),
    Dense(128, 64, relu),
    Dense(64, 5),
    softmax
) |> device

# Define the loss function
loss(model, X, y) = Flux.crossentropy(model(X), y)

# Setup the optimizer
opt = Flux.setup(NADAM(0.0004), model)

# Function to calculate accuracy
accuracy(X, y) = mean(onecold(model(X)) .== onecold(y))

# Training with validation
best_val_accuracy = 0.0
best_model_params = nothing
last_imp=0
for epoch in 1:491
    Flux.train!(loss, model, loader, opt)
    val_accuracy = accuracy(X_valid1, y_valid1)
    println("Epoch $epoch: Validation Accuracy = $val_accuracy")

    if val_accuracy > best_val_accuracy
        best_val_accuracy = val_accuracy
        model_params = cpu(Flux.state(model))
        BSON.@save "bestmlp3layersownnestor.bson" model_params
       last_imp=0
    end
    if last_imp>10
        break
    end
end



# Evaluate on the test set
test_accuracy = accuracy(X_test1, y_test1)
println("MLP Accuracy on Test Set: ", test_accuracy)


[ Info: Training on GPU


Epoch 1: Validation Accuracy = 0.47375
Epoch 2: Validation Accuracy = 0.47375
Epoch 3: Validation Accuracy = 0.47375
Epoch 4: Validation Accuracy = 0.48291666666666666
Epoch 5: Validation Accuracy = 0.4945833333333333
Epoch 6: Validation Accuracy = 0.50875
Epoch 7: Validation Accuracy = 0.51375
Epoch 8: Validation Accuracy = 0.5241666666666667
Epoch 9: Validation Accuracy = 0.5345833333333333
Epoch 10: Validation Accuracy = 0.515
Epoch 11: Validation Accuracy = 0.5341666666666667
Epoch 12: Validation Accuracy = 0.5545833333333333
Epoch 13: Validation Accuracy = 0.5416666666666666
Epoch 14: Validation Accuracy = 0.5495833333333333
Epoch 15: Validation Accuracy = 0.5541666666666667
Epoch 16: Validation Accuracy = 0.5554166666666667
Epoch 17: Validation Accuracy = 0.5316666666666666
Epoch 18: Validation Accuracy = 0.5666666666666667
Epoch 19: Validation Accuracy = 0.56875
Epoch 20: Validation Accuracy = 0.56625
Epoch 21: Validation Accuracy = 0.56375
Epoch 22: Validation Accuracy = 0.5637

In [8]:


# Check if GPU is available and set the device accordingly
use_cuda = true
if use_cuda && CUDA.functional()
    device = gpu
    @info "Training on GPU"
else
    device = cpu
    @info "Training on CPU"
end

# Transpose and move X_train to the correct device
X_train1 = transpose(X_train) |> device
batch_size = 32
X_valid1 = transpose(X_valid) |> device
X_test1 = transpose(X_test) |> device

# Convert y_train to onehot encoding and move to the correct device
y_train1 = Flux.onehotbatch(y_train, 1:5) |> device
y_valid1 = Flux.onehotbatch(y_valid, 1:5) |> device
y_test1 = Flux.onehotbatch(y_test, 1:5) |> device

# Create DataLoader and move to the correct device
loader = Flux.DataLoader((X_train1, y_train1), batchsize=batch_size, shuffle=true)

# Define the MLP model and move to the correct device
model= Chain(
  Dense(300 => 32, relu),
  Dense(32 => 5),
  softmax) |> device

# Define the loss function
loss(model, X, y) = Flux.crossentropy(model(X), y)

# Setup the optimizer
opt = Flux.setup(Adam(0.0004), model)

# Function to calculate accuracy
accuracy(X, y) = mean(onecold(model(X)) .== onecold(y))

# Training with validation
best_val_accuracy = 0.0
best_model_params = nothing
last_imp=0
for epoch in 1:491
    Flux.train!(loss, model, loader, opt)
    val_accuracy = accuracy(X_valid1, y_valid1)
    println("Epoch $epoch: Validation Accuracy = $val_accuracy")

    if val_accuracy > best_val_accuracy
        best_val_accuracy = val_accuracy
        model_params = cpu(Flux.state(model))
        BSON.@save "simpleownadam.bson" model_params 
         last_imp=0
    end
    if last_imp>10
        break
    end
end




# Evaluate on the test set
test_accuracy = accuracy(X_test1, y_test1)
println("MLP Accuracy on Test Set: ", test_accuracy)


[ Info: Training on GPU


Epoch 1: Validation Accuracy = 0.47375
Epoch 2: Validation Accuracy = 0.47375
Epoch 3: Validation Accuracy = 0.47375
Epoch 4: Validation Accuracy = 0.47375
Epoch 5: Validation Accuracy = 0.47375
Epoch 6: Validation Accuracy = 0.47375
Epoch 7: Validation Accuracy = 0.47375
Epoch 8: Validation Accuracy = 0.47375
Epoch 9: Validation Accuracy = 0.47375
Epoch 10: Validation Accuracy = 0.47375
Epoch 11: Validation Accuracy = 0.47375
Epoch 12: Validation Accuracy = 0.4741666666666667
Epoch 13: Validation Accuracy = 0.4741666666666667
Epoch 14: Validation Accuracy = 0.47791666666666666
Epoch 15: Validation Accuracy = 0.48083333333333333
Epoch 16: Validation Accuracy = 0.47708333333333336
Epoch 17: Validation Accuracy = 0.49541666666666667
Epoch 18: Validation Accuracy = 0.49416666666666664
Epoch 19: Validation Accuracy = 0.4925
Epoch 20: Validation Accuracy = 0.4979166666666667
Epoch 21: Validation Accuracy = 0.5
Epoch 22: Validation Accuracy = 0.49666666666666665
Epoch 23: Validation Accuracy

In [7]:
using Flux
using Flux: onehot, onecold, onehotbatch, crossentropy, throttle, params
using Zygote
using BSON, JLD2, Statistics
using CUDA
using Random
using Base.Iterators: repeated, partition
import StatsBase.sample, StatsBase.Weights

# Check if GPU is available and set the device accordingly
use_cuda = true
if use_cuda && CUDA.functional()
    device = gpu
    @info "Training on GPU"
else
    device = cpu
    @info "Training on CPU"
end

# Transpose and move X_train to the correct device
X_train1 = transpose(X_train) |> device
batch_size = 32
X_valid1 = transpose(X_valid) |> device
X_test1 = transpose(X_test) |> device

# Convert y_train to onehot encoding and move to the correct device
y_train1 = Flux.onehotbatch(y_train, 1:5) |> device
y_valid1 = Flux.onehotbatch(y_valid, 1:5) |> device
y_test1 = Flux.onehotbatch(y_test, 1:5) |> device

# Create DataLoader and move to the correct device
loader = Flux.DataLoader((X_train1, y_train1), batchsize=batch_size, shuffle=true)

# Define the MLP model and move to the correct device
model= Chain(
  Dense(300 => 32, relu),
  Dense(32 => 5),
  softmax) |> device

# Define the loss function
loss(model, X, y) = Flux.crossentropy(model(X), y)

# Setup the optimizer
opt = Flux.setup(AdaGrad(0.04), model)

# Function to calculate accuracy
accuracy(X, y) = mean(onecold(model(X)) .== onecold(y))

# Training with validation
best_val_accuracy = 0.0
best_model_params = nothing
last_imp=0
for epoch in 1:491
    Flux.train!(loss, model, loader, opt)
    val_accuracy = accuracy(X_valid1, y_valid1)
    println("Epoch $epoch: Validation Accuracy = $val_accuracy")

    if val_accuracy > best_val_accuracy
        best_val_accuracy = val_accuracy
        model_params = cpu(Flux.state(model))
        BSON.@save "simpleadaown.bson" model_params 
        last_imp=0
    end
    if last_imp>10
        break
    end
end



# Evaluate on the test set
test_accuracy = accuracy(X_test1, y_test1)
println("MLP Accuracy on Test Set: ", test_accuracy)


[ Info: Training on GPU


Epoch 1: Validation Accuracy = 0.47375
Epoch 2: Validation Accuracy = 0.47375
Epoch 3: Validation Accuracy = 0.47375
Epoch 4: Validation Accuracy = 0.47375
Epoch 5: Validation Accuracy = 0.47375
Epoch 6: Validation Accuracy = 0.47375
Epoch 7: Validation Accuracy = 0.47375
Epoch 8: Validation Accuracy = 0.47375
Epoch 9: Validation Accuracy = 0.47375
Epoch 10: Validation Accuracy = 0.47375
Epoch 11: Validation Accuracy = 0.47583333333333333
Epoch 12: Validation Accuracy = 0.47583333333333333
Epoch 13: Validation Accuracy = 0.47583333333333333
Epoch 14: Validation Accuracy = 0.48541666666666666
Epoch 15: Validation Accuracy = 0.47791666666666666
Epoch 16: Validation Accuracy = 0.48583333333333334
Epoch 17: Validation Accuracy = 0.48083333333333333
Epoch 18: Validation Accuracy = 0.48333333333333334
Epoch 19: Validation Accuracy = 0.49875
Epoch 20: Validation Accuracy = 0.49666666666666665
Epoch 21: Validation Accuracy = 0.49416666666666664
Epoch 22: Validation Accuracy = 0.496666666666666